In [1]:
!pip install polars > istallations.txt

In [2]:
import polars as pl
import numpy as np
import pandas as pd
import os
import time
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/kaggle/input/sbermarket-internship-competition/sample_submission.csv
/kaggle/input/sbermarket-internship-competition/train.csv


In [3]:
raw = pl.read_csv('../input/sbermarket-internship-competition/train.csv')
sub = pl.read_csv('../input/sbermarket-internship-competition/sample_submission.csv')


In [97]:

raw = raw.with_columns(raw['cart'].cast(str).alias("cart"))
train_raw = raw.to_dummies(columns = 'cart')
train_raw = train_raw.group_by(['user_id', 'order_completed_at']).max()
train_raw = train_raw.sort(['user_id', 'order_completed_at'])
train_raw = train_raw.with_columns(
    (pl.col('user_id').cum_count() - pl.lit(1)).over(['user_id']).alias('order_number')
)
train_raw = train_raw.drop('order_completed_at')

grouped = train_raw.group_by('user_id').agg(pl.max('order_number').alias('max_order_number'))
merged = train_raw.join(grouped, on='user_id')
last_order = merged['max_order_number'] == merged['order_number']
train_data = train_raw.filter(~last_order).group_by('user_id').sum()
valid_data = train_raw.filter(last_order)

train_melt = train_data.drop('order_number').melt(id_vars=['user_id'], variable_name='category', value_name='ordered')
valid_melt = valid_data.drop('order_number').melt(id_vars=['user_id'], variable_name='category', value_name='ordered')
Train = train_melt

order_number_df = valid_data.select(['user_id', 'order_number']).unique()
Train = Train.join(order_number_df, on='user_id', how='left')
Train = Train.with_columns((pl.col('ordered') / pl.col('order_number')).alias('rating'))

user_id,cart_0,cart_1,cart_10,cart_100,cart_101,cart_102,cart_103,cart_104,cart_105,cart_106,cart_107,cart_108,cart_109,cart_11,cart_110,cart_111,cart_112,cart_113,cart_114,cart_115,cart_116,cart_117,cart_118,cart_119,cart_12,cart_120,cart_121,cart_122,cart_123,cart_124,cart_125,cart_126,cart_127,cart_128,cart_129,cart_13,…,cart_86,cart_860,cart_861,cart_862,cart_863,cart_864,cart_865,cart_866,cart_867,cart_868,cart_869,cart_87,cart_870,cart_871,cart_872,cart_873,cart_874,cart_875,cart_876,cart_877,cart_878,cart_879,cart_88,cart_880,cart_89,cart_9,cart_90,cart_91,cart_92,cart_93,cart_94,cart_95,cart_96,cart_97,cart_98,cart_99,order_number
i64,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,…,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u32
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,3
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,4
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5


In [104]:
valid_data

user_id,cart_0,cart_1,cart_10,cart_100,cart_101,cart_102,cart_103,cart_104,cart_105,cart_106,cart_107,cart_108,cart_109,cart_11,cart_110,cart_111,cart_112,cart_113,cart_114,cart_115,cart_116,cart_117,cart_118,cart_119,cart_12,cart_120,cart_121,cart_122,cart_123,cart_124,cart_125,cart_126,cart_127,cart_128,cart_129,cart_13,…,cart_86,cart_860,cart_861,cart_862,cart_863,cart_864,cart_865,cart_866,cart_867,cart_868,cart_869,cart_87,cart_870,cart_871,cart_872,cart_873,cart_874,cart_875,cart_876,cart_877,cart_878,cart_879,cart_88,cart_880,cart_89,cart_9,cart_90,cart_91,cart_92,cart_93,cart_94,cart_95,cart_96,cart_97,cart_98,cart_99,order_number
i64,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,…,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u32
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,8
2,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,14
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,6
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,7
5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,14
6,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,9
7,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,10
8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7


In [96]:
%%time
# sparse matrix for temporary use
train_raw = pd.get_dummies(raw_pd, columns = ['cart'], prefix='', prefix_sep='', dtype='bool')
train_raw = train_raw.groupby(['user_id', 'order_completed_at']).any().reset_index()

# order counter for each use
train_raw['order_number'] = train_raw.groupby(['user_id']).cumcount()
train_raw = train_raw.drop('order_completed_at', axis=1)

# separate datasets by the last purchase
last_order = train_raw.groupby(['user_id'])['order_number'].transform(max) == train_raw['order_number']
train = train_raw[~last_order].groupby('user_id').sum().reset_index()
valid = train_raw[last_order].reset_index(drop=True)

#purchase counter for each category for each user
train_melt = pd.melt(train, id_vars=['user_id'], var_name='category', value_name='ordered')
valid_melt = pd.melt(valid, id_vars=['user_id'], var_name='category', value_name='target')

# Train = train_melt.copy()

# # total purchase counter for each user
# order_number = valid[['user_id', 'order_number']].set_index('user_id').squeeze()
# Train['orders_total']= Train['user_id'].map(order_number)

# #average amount of each category in  customer's purchase
# Train['rating'] = Train['ordered'] / Train['orders_total']

# # user_id / category as in submission file
# Train['id'] = Train['user_id'].astype(str) + ';' + Train['category']

# # target variable (the last known purchase)
# Train['target'] = valid_melt['target'].astype(int)

# #remove those users/categories who are not represented in the submission file
# Train = Train[Train.id.isin(sub.id.unique())].reset_index(drop=True)
# #Check
# print((Train.sort_values('id')['id'].values == Train.sort_values('id')['id'].values).all())

# #purchase counter by all user (for represetned users)
# total_ordered = Train.groupby('category')['ordered'].sum()
# Train['total_ordered'] = Train['category'].map(total_ordered)

# Train.head(3)

train_melt

<timed exec>:10: FutureWarning: The provided callable <built-in function max> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.


CPU times: user 53.9 s, sys: 13.2 s, total: 1min 7s
Wall time: 1min 7s


,user_id,category,ordered
0,0,0,0
1,1,0,0
2,2,0,1
3,3,0,0
4,4,0,0
...,...,...,...
17639995,19995,order_number,1
17639996,19996,order_number,1
17639997,19997,order_number,1
17639998,19998,order_number,1


In [16]:
train

,user_id,0,1,2,3,4,5,6,7,8,...,872,873,874,875,876,877,878,879,880,order_number
0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,28
2,2,1,0,0,0,0,2,0,0,0,...,0,0,0,0,0,0,0,0,0,91
3,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,15
4,4,0,0,0,0,0,2,0,0,0,...,0,0,0,0,0,0,0,0,0,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,19995,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
19996,19996,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
19997,19997,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
19998,19998,0,0,0,0,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,1
